<a href="https://colab.research.google.com/github/rifrafff/DERS4/blob/master/Do%C4%9Fal_Dil_%C4%B0%C5%9Fleme_%C3%96dev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install simplemma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.5/75.5 MB 6.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import spacy
from simplemma import text_lemmatizer
import numpy as np
import ast
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
df = pd.read_excel("kitap.xlsx")
df.head()

,Kitap,Konu
0,Kürk Mantolu Madonna,"Her gün, daima öğleden sonra oraya gidiyor, ko..."
1,Şeker Portakalı,"Yazarlıkta karar kılıncaya kadar, boks antrenö..."
2,Hayvan Çiftliği,"İngiliz yazar George Orwell (1903-1950), ülkem..."
3,Satranç,"Satranç sonsuz eski, ama aynı zamanda sonrasız..."
4,Simyacı,"Simyacı, dünyaca ünlü Brezilyalı yazar Paulo C..."


In [ ]:
target_column = 'Konu'

# Stop words listesi
stop_words = set(stopwords.words('turkish'))

In [ ]:
def remove_stopwords(text):
    word_tokens = word_tokenize(text)
    filtered_text = [word for word in word_tokens if word.lower() not in stop_words]
    return ' '.join(filtered_text)

In [ ]:
new_column = 'temizlenmis_konu'

# Stop words'leri temizle ve yeni sütuna ekle
df[new_column] = df[target_column].apply(remove_stopwords)

In [ ]:
df.head()

,Kitap,Konu,temizlenmis_konu
0,Kürk Mantolu Madonna,"Her gün, daima öğleden sonra oraya gidiyor, ko...","gün , daima öğleden sonra oraya gidiyor , kori..."
1,Şeker Portakalı,"Yazarlıkta karar kılıncaya kadar, boks antrenö...","Yazarlıkta karar kılıncaya kadar , boks antren..."
2,Hayvan Çiftliği,"İngiliz yazar George Orwell (1903-1950), ülkem...","İngiliz yazar George Orwell ( 1903-1950 ) , ül..."
3,Satranç,"Satranç sonsuz eski, ama aynı zamanda sonrasız...","Satranç sonsuz eski , aynı zamanda sonrasız ye..."
4,Simyacı,"Simyacı, dünyaca ünlü Brezilyalı yazar Paulo C...","Simyacı , dünyaca ünlü Brezilyalı yazar Paulo ..."


In [ ]:
# stop word ayıklaması yapmayan metin temizleme fonk.
def clean_text(text):
    text = text.replace("Â", "a")
    text = text.replace("â", "a")
    text = text.replace("î", "i")
    text = text.replace("Î", "ı")
    text = text.replace("İ", "i")
    text = text.replace("I", "ı")
    text = text.replace(u"\u00A0", " ")
    text = text.replace("|", " ")

    text = re.sub(r"@[A-Za-z0-9]+", " ", text)
    text = re.sub(r"(.)\1+", r"\1\1", text)
    text = re.sub(r"https?:\/\/\S+", " ", text)
    text = re.sub(r"http?:\/\/\S+", " ", text)
    text = re.sub(r"\n", " ", text)
    text = re.sub(r"#(\w+)", " ", text)
    text = re.sub(r"^\x00-\x7F]+", " ", text)
    text = re.sub(r"[^A-Za-zâîığüşöçİĞÜŞÖÇ]+", " ", text)
    text = re.sub(r"((https://[^\s]+))", " ", text)
    text = " ".join(text.lower().strip().split())

    return " ".join(text_lemmatizer(text, lang="tr"))

In [ ]:
def clean_row(row):
    konu = clean_text(row["temizlenmis_konu"])
    return konu

In [ ]:
clean_row(df.iloc[1])

'yazar karar kıl kadar boks antrenörlük ressam heykeltıraş model yap muz plantasyon hamallık gece kulüp garsonluk kadar çeşit iş çalış jose mauro vasconcelos başyapıt şeker portakal gün biri acı keşfet küçük bir çocuk öykü dür yoksul bir aile oğul ol dünya gel dokuz yaş yüz öğren bir gün yüz şampiyon ol hayal kur vasconcelos çocukluk derin izle taşı şeker portakal yaşam bekle değişim karşı büyük sarsıntı yaşa küçük zezenin baş geçen anlat vasconcelos tam on iki gün yaz roman yirmi yıl fazla bir zaman yürek taşı söyle aydın emeçin güzel türkçe dil armağan et şeker portakal başkahraman zezenin büyü yaşa serüven yazar güneş uyan delifişek ad roman izle'

In [ ]:
df["clean"] = df.apply(lambda row: clean_row(row), axis=1)
df.head()
df.to_excel("guncellenmis_veri_seti.xlsx", index=False)

In [ ]:
features = df['clean'].values
labels = df['Kitap'].values

In [ ]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(features)

In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
kitap_yorumlari_tokenized = [nlp(yorum.lower()) for yorum in df['clean']]

In [ ]:
def konu_yaz(konu):
  kullanici_konu_tokenized = nlp(konu.lower())
  kullanici_vektor = vectorizer.transform([konu.lower()])
  benzerlik_skorlari = linear_kernel(kullanici_vektor, X).flatten()
  en_yakin_kitap_index = benzerlik_skorlari.argmax()
  onerilen_kitap = df['Kitap'][en_yakin_kitap_index]

  return onerilen_kitap,benzerlik_skorlari

In [ ]:
print(konu_yaz("silah"))

('Silahlara Veda', array([0., 0., 0., ..., 0., 0., 0.]))


In [ ]:
from sklearn.model_selection import train_test_split

# Veriyi eğitim ve test setlere ayır
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

In [ ]:
correct_predictions = 0

for i, test_konu in enumerate(X_test):
    predicted_kitap = konu_yaz(test_konu)
    actual_kitap = y_test[i]

    if predicted_kitap == actual_kitap:
        correct_predictions += 1

accuracy = correct_predictions / len(X_test)
print(f"Başarı Oranı: {accuracy * 100:.2f}%")

Başarı Oranı: 0.00%


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Veriyi eğitim ve test setlere ayır
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

correct_predictions = 0

for i, test_konu in enumerate(X_test):
    predicted_kitap = konu_yaz(test_konu)
    actual_kitap = y_test[i]

    if predicted_kitap == actual_kitap:
        correct_predictions += 1

accuracy = correct_predictions / len(X_test)
print(f"Doğruluk (Accuracy): {accuracy * 100:.2f}%")

Doğruluk (Accuracy): 0.00%


In [ ]:
def konu_bul(konu):
    kullanici_konu_tokenized = nlp(konu.lower())
    kullanici_vektor = vectorizer.transform([konu.lower()])
    benzerlik_skorlari = linear_kernel(kullanici_vektor, X).flatten()

    benzerlik_kitap_indeksleri = list(enumerate(benzerlik_skorlari))

    sirali_benzerlikler = sorted(benzerlik_kitap_indeksleri, key=lambda x: x[1], reverse=True)

    en_yakin_kitaplar = []
    for i in range(min(5, len(sirali_benzerlikler))):
        indeks, benzerlik_skoru = sirali_benzerlikler[i]
        onerilen_kitap = df['Kitap'][indeks]
        en_yakin_kitaplar.append((onerilen_kitap, benzerlik_skoru))

    # En yüksek benzerlik oranına sahip olan kitabın indeksini bulma
    en_yuksek_benzerlik_indeks = sirali_benzerlikler[0][0]

    # En yüksek benzerlik oranına sahip olan kitabın konusunu alma
    en_yuksek_benzerlik_kitap_konu = df['clean'][en_yuksek_benzerlik_indeks]

    return en_yakin_kitaplar, en_yuksek_benzerlik_kitap_konu

# Örnek kullanım
kullanici_konu = "rüya evreni"
en_yakin_kitaplar, en_yuksek_benzerlik_kitap_konu = konu_bul(kullanici_konu)


# Diğer en yüksek benzerlik oranına sahip olan kitapları ve benzerlik oranlarını bastırma
print("\nEn Yüksek Benzerlik Oranları:")
for kitap, benzerlik_orani in en_yakin_kitaplar:
    print(f"{kitap}: Benzerlik Oranı = {benzerlik_orani:.4f}")


En Yüksek Benzerlik Oranları:
Başka Bir Şey: Benzerlik Oranı = 0.6013
Veliaht - 1: Benzerlik Oranı = 0.3586
Size Bir Sır Vereceğim: Benzerlik Oranı = 0.3520
Karanlıktan Sonra: Benzerlik Oranı = 0.3252
Bir Yaz Gecesi Rüyası: Benzerlik Oranı = 0.2864


In [ ]:
pip install transformers

In [ ]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
pip install tensorflow tensorflow-hub

In [ ]:
pip install tensorflow-text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 11.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import tensorflow as tf
import tensorflow_text
import tensorflow_hub as hub
from sklearn.metrics.pairwise import cosine_similarity

# Veri setini yükle
veri_seti = pd.read_excel("guncellenmis_veri_seti.xlsx")  # Excel dosyanızın adını güncelleyin.

# Boş değerleri temizle
veri_seti = veri_seti.dropna()

use_model = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

konu_embeddings = use_model(veri_seti['clean'])

# Kitap önerisi fonksiyonu
def kitap_onerisi_use(konu):
    konu_embedding = use_model([konu])

    # Cosine Similarity skorlarını hesapla
    benzerlik_skorları = cosine_similarity(konu_embedding, konu_embeddings).flatten()

    # Benzerlik skorlarına göre sırala
    benzer_kitaplar = (-benzerlik_skorları).argsort()[:5]

    # En yakın 5 kitabı ve benzerlik skorlarını yazdır
    print("\nEn Yakın 5 Kitap:")
    for i, kitap_index in enumerate(benzer_kitaplar):
        kitap_isim = veri_seti['Kitap'][kitap_index]
        benzerlik_skoru = benzerlik_skorları[kitap_index]
        print(f"{i + 1}. {kitap_isim} - Benzerlik Skoru: {benzerlik_skoru}")

    # Kİtap isimlerini döndür
    kitap_isimleri = [veri_seti['Kitap'][i] for i in benzer_kitaplar]

    return kitap_isimleri

# Örnek kullanım
kullanici_konu = "rüya evreni"  # Kendi konunuzu girin
onerilen_kitaplar = kitap_onerisi_use(kullanici_konu)


En Yakın 5 Kitap:
1. Daktiloya Çekilmiş Şiirler (1977-1987) - Benzerlik Skoru: 0.21017774939537048
2. Kendine Hoş Geldin - Benzerlik Skoru: 0.12206152081489563
3. Bu Bizim Hikayemiz - Benzerlik Skoru: 0.11782137304544449
4. Dünyalar Savaşı - Benzerlik Skoru: 0.11639043688774109
5. Leyla ile Mecnun - Benzerlik Skoru: 0.11602406203746796


In [ ]:
import pandas as pd
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import word_tokenize

# Veri setini yükle
veri_seti = pd.read_excel("guncellenmis_veri_seti.xlsx")

# Boş değerleri temizle
veri_seti = veri_seti.dropna()

# NLTK'nin gereksinim duyduğu kaynakları indir
import nltk
nltk.download('punkt')

# Veri setindeki konuları belirli bir formata getirerek kullanmamız gerekiyor
tagged_data = [TaggedDocument(words=word_tokenize(konu.lower()), tags=[str(i)]) for i, konu in enumerate(veri_seti['clean'])]

# Doc2Vec modelini eğitiyoruz
model = Doc2Vec(vector_size=100, window=2, min_count=1, workers=4, epochs=100)
model.build_vocab(tagged_data)
model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)

# kitap önerisi fonksiyonu
def kitap_onerisi_doc2vec(konu):
    konu_vector = model.infer_vector(word_tokenize(konu.lower()))

    # Tüm kitap vektörlerini al
    kitap_vektorleri = [model.dv[i] for i in range(len(veri_seti))]

    # Cosine Similarity skorlarını hesapla
    benzerlik_skorları = cosine_similarity([konu_vector], kitap_vektorleri).flatten()

    # Benzerlik skorlarına göre sırala
    benzer_kitaplar = (-benzerlik_skorları).argsort()[:5]

    # En yakın 5 kitabı ve benzerlik skorlarını yazdır
    print("\nEn Yakın 5 Kitap:")
    for i, kitap_index in enumerate(benzer_kitaplar):
        kitap_isim = veri_seti['Kitap'][kitap_index]
        benzerlik_skoru = benzerlik_skorları[kitap_index]
        print(f"{i + 1}. {kitap_isim} - Benzerlik Skoru: {benzerlik_skoru}")

    # kitap isimlerini döndür
    kitap_isimleri = [veri_seti['Kitap'][i] for i in benzer_kitaplar]

    return kitap_isimleri


kullanici_konu = "rüya evreni"  # Kendi konunuzu girin
onerilen_kitaplar = kitap_onerisi_doc2vec(kullanici_konu)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!



En Yakın 5 Kitap:
1. Daktiloya Çekilmiş Şiirler (1977-1987) - Benzerlik Skoru: 0.6593515872955322
2. Size Bir Sır Vereceğim - Benzerlik Skoru: 0.5791665315628052
3. Başka Bir Şey - Benzerlik Skoru: 0.5740090012550354
4. Ev Sahibesi - Benzerlik Skoru: 0.5691208243370056
5. Beyaz Gemi - Benzerlik Skoru: 0.5548527836799622
